In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importing required packages

import pandas as pd
import numpy as np

import os
import gc

from ta_lib.data_utils.data_processing import get_aggregated_data
from ta_lib.core.api import load_yml, create_context

In [3]:
# Defining configs

config_path = '../conf/config.yml'
config = create_context(config_path)


In [4]:
config.offline

{'CREDENTIALS': {'db': {'redshift': {'dbname': 'gdap',
    'host': 'GDAP_GDW.mcdonalds.com',
    'port': 5439,
    'username': 'mca0052',
    'password': 'Y&W$Up0z+wropE',
    'uri': 'dbname={dbname}  host={host}  port={port} user={username} password={password}'},
   'clickhouse': {'dbname': 'crm_analitycs_new',
    'host': '10.144.0.101',
    'port': 9000,
    'username': 'tiger',
    'password': 'Xs8ZwQ3KuRXhyEf4',
    'uri': 'clickhouse+native://{username}:{password}@{host}:{port}/{dbname}'}}},
 'SOURCE_DIR': 'G:\\\\Shared drives\\\\McD Discount Engine\\\\Russia\\\\1. Data\\\\',
 'SOURCE_DIGITAL_PROMOTION_PATH': '{SOURCE_DIR}digital\\\\digital_offers\\\\cleaned\\\\Digital offers_support with placement.xlsx',
 'PROCESSED_DIGITAL_PROMOTION_PATH': '{SOURCE_DIR}digital\\\\digital_offers\\\\processed\\\\promotion_data_placement.csv',
 'PROCESSED_DIGITAL_CITIES_PATH': '{SOURCE_DIR}digital\\\\digital_offers\\\\processed\\\\cities.csv',
 'SOURCE_TM_PROMOTION_PATH': '{SOURCE_DIR}offline\\\\o

In [5]:
# Getting Missing data
import multiprocessing as mlp
import time

In [6]:
dates_=sorted(pd.date_range('2018-01-01','2021-08-01',freq='MS'),reverse=True)
dates_=[i.strftime('%Y%m%d') for i in dates_]

In [10]:
import time
t1 = time.time()
get_aggregated_data.get_ungrouped_data(cfg.offline,'month_id=20210801', 'both', True, True, False,True)
print("time taken ",(time.time()-t1)/60)

month_id=20210801
abc
<class 'AttributeError'> get_aggregated_data.py 1056
time taken  1.1990222334861755


In [42]:
cfg = config.offline
file = 'month_id=20210801'
result_filter='both'
day_level=True
item_level=True
get_price_info=True
include_105=True

try:
    offline_input_path = cfg['OFFLINE_RAW'].format(**cfg)
    offline_input_path_105 = cfg['OFFLINE_RAW_105'].format(**cfg)
    digital_input_path = cfg['DIGITAL_TRNS'].format(**cfg)
    print(file)
    print("abc")

    # Getting file list and initialising other variables
#     file_list = [x for x in os.listdir(offline_input_path)]
#     file_list.sort(reverse=True)

#     print(file_list)

    grouper_cols = ['sld_menu_itm_id', 'pos_busn_dt']
    grouper_cols_day = ['pos_busn_dt']


# Looping over month parquet and aggregating

    # Loading dumped GDW data

    df = pd.read_parquet(offline_input_path + file + '/data.parquet',columns=['pos_busn_dt', 
                                  'mcd_gbal_lcat_id_nu', 
                                  'pos_ord_nu', 
                                  'cmbo_pren_sld_menu_itm_id','pos_itm_grss_unt_prc_am','sld_menu_itm_id','terr_pos_trn_typ_cd','pos_itm_tot_qt']).query('sld_menu_itm_id>=1000 and sld_menu_itm_id<10000').head(100)
    df['sld_menu_itm_id'] = df['sld_menu_itm_id'].astype('int')

    if include_105:
        df_105 = pd.read_parquet(offline_input_path_105 + file + '/data.parquet',columns=['pos_busn_dt', 
                                  'mcd_gbal_lcat_id_nu', 
                                  'pos_ord_nu', 
                                  'cmbo_pren_sld_menu_itm_id','pos_itm_grss_unt_prc_am','sld_menu_itm_id','terr_pos_trn_typ_cd','pos_itm_tot_qt']).query('sld_menu_itm_id>=1000 and sld_menu_itm_id<10000').head(100)
        df_105['sld_menu_itm_id'] = df_105['sld_menu_itm_id'].astype('int')
        df = df.append(df_105)

    # Doing processing to get wap and mode price
    # Separating alc and Combo trns to get mode price
    alc_df = df[(df['cmbo_pren_sld_menu_itm_id'] == -1)].reset_index(drop=True)
    cmbo_df = df[(df['cmbo_pren_sld_menu_itm_id'] != -1)].reset_index(drop=True)

    del df

    cmbo_pr_df = cmbo_df.groupby(['pos_busn_dt', 
                                  'mcd_gbal_lcat_id_nu', 
                                  'pos_ord_nu', 
                                  'cmbo_pren_sld_menu_itm_id'])[['pos_itm_grss_unt_prc_am']].sum().reset_index()

    ran_df = cmbo_df[(cmbo_df['cmbo_pren_sld_menu_itm_id'] ==   cmbo_df['sld_menu_itm_id'])].groupby(['pos_busn_dt', 
                                                                                             'mcd_gbal_lcat_id_nu', 
                                                                                             'pos_ord_nu', 
                                                                                             'cmbo_pren_sld_menu_itm_id'])[['pos_itm_tot_qt']].sum().reset_index()

    cmbo_pr_df = cmbo_pr_df.merge(ran_df, on=['pos_busn_dt', 'mcd_gbal_lcat_id_nu', 'pos_ord_nu', 
                                              'cmbo_pren_sld_menu_itm_id'], how='left')
    cmbo_pr_df['pos_itm_tot_qt'].fillna(1, inplace=True)
    cmbo_pr_df['cmbo_price'] = cmbo_pr_df['pos_itm_grss_unt_prc_am']/cmbo_pr_df['pos_itm_tot_qt']

    del cmbo_pr_df['pos_itm_grss_unt_prc_am']
    del cmbo_pr_df['pos_itm_tot_qt']

    cmbo_df = cmbo_df.merge(cmbo_pr_df, on=['pos_busn_dt', 'mcd_gbal_lcat_id_nu', 'pos_ord_nu', 
                                            'cmbo_pren_sld_menu_itm_id'], how='left')
    cmbo_df['pos_itm_grss_unt_prc_am'] = cmbo_df['cmbo_price']        
    del cmbo_df['cmbo_price']

    # Cncatenating the dataframe and removing child items for further processing
    df = pd.concat([alc_df, cmbo_df], axis=0).reset_index(drop=True)

    df = df.query('(cmbo_pren_sld_menu_itm_id == -1) or (cmbo_pren_sld_menu_itm_id == sld_menu_itm_id)').reset_index(drop=True)


    # Loading digital trn data if exist and doing separate operations
    digital_flag = 0
    if os.path.exists(digital_input_path + file + '/'):

        # Loading digital data
        dig_df =  pd.read_parquet(digital_input_path + file + '/part-0.parquet',columns=['pos_ord_nu', 'mcd_gbal_lcat_id_nu']).head(100)
        print(dig_df.shape)

        dig_df['digital_flag'] = 1

        # Merging digital data
        df = df.merge(dig_df, on=['mcd_gbal_lcat_id_nu', 'pos_ord_nu'], how='left')

        # Clearing space
        del dig_df
        digital_flag = 1

    else :
        df['digital_flag'] = np.NaN


    if ('digital' in result_filter or 'all' in result_filter or 'both' in result_filter) and digital_flag == 1 :

        # Defining variable paths
        digital_output_path = cfg['UNITS'].format(**cfg, type_='digital')
        digital_day_output_path = cfg['DAY_UNITS'].format(**cfg, type_='digital')
        digital_wap_output_path = cfg['WAP'].format(**cfg, type_='digital')

        # Selecting digital trns and getting WAP and Mode data 
        dig_temp_df = df.query('digital_flag.notna()').reset_index(drop=True)
        dig_temp_df['mult'] = dig_temp_df.eval('pos_itm_grss_unt_prc_am * pos_itm_tot_qt')

        if get_price_info :

            # Getting WAP
            dig_wap_df = dig_temp_df.groupby(grouper_cols)[['mult', 'pos_itm_tot_qt']].sum().reset_index()
            dig_wap_df['WAP'] = (dig_wap_df['mult']/dig_wap_df['pos_itm_tot_qt']).round(2)

            # Getting Mode
            dig_md_df = dig_temp_df.groupby(grouper_cols + ['pos_itm_grss_unt_prc_am'])[['pos_itm_tot_qt']].sum().reset_index()
            dig_md_df['rank'] = dig_md_df.groupby(grouper_cols)[['pos_itm_tot_qt']].rank(ascending=False)['pos_itm_tot_qt'].values
            dig_md_df = dig_md_df[(dig_md_df['rank'] == 1)].reset_index(drop=True)
            dig_md_df['mult'] = dig_md_df['pos_itm_grss_unt_prc_am']*dig_md_df['pos_itm_tot_qt']

            dig_md_df_res = dig_md_df.groupby(grouper_cols)[['mult', 'pos_itm_tot_qt']].sum().reset_index()
            dig_md_df_res['Mode_price'] = dig_md_df_res['mult']/dig_md_df_res['pos_itm_tot_qt']

            del dig_wap_df['mult']
            del dig_wap_df['pos_itm_tot_qt']

            del dig_md_df_res['mult']
            del dig_md_df_res['pos_itm_tot_qt']

            # Doing processing and storing data
            dig_wap_df = dig_wap_df.merge(dig_md_df_res, on=grouper_cols, how='left')  

            # create dir if doesn't exist 
            os.makedirs(digital_wap_output_path, exist_ok=True)

            dig_wap_df = format_dtype(dig_wap_df, add_monthid=True)

            dig_wap_df.to_parquet(digital_wap_output_path, partition_cols=['sld_menu_itm_id', 'monthid'])
            _change_permissions_recursive(digital_wap_output_path, 0o777)

            # Clearing spaces
            del dig_md_df
            del dig_wap_df            

        if item_level :

            # Getting Units and GC data
            dig_res_df = dig_temp_df.groupby(grouper_cols)[['pos_itm_tot_qt']].sum().reset_index()
            dig_res_df = dig_res_df.merge(dig_temp_df.groupby(grouper_cols)[['terr_pos_trn_typ_cd',
                                                                        'pos_busn_dt',
                                                                        'mcd_gbal_lcat_id_nu',
                                                                        'pos_ord_nu']].apply(add_gc_count).reset_index(), 
                                      on=grouper_cols, how='outer')

            # create dir if donesn't exist 
            os.makedirs(digital_output_path, exist_ok=True)

            # Doing processing and storing result df
            dig_res_df.rename(columns={0:'num_trans', 'pos_itm_tot_qt':'quantity'}, inplace=True)

            dig_res_df = format_dtype(dig_res_df, add_monthid=True)

            dig_res_df.to_parquet(digital_output_path, partition_cols=['sld_menu_itm_id', 'monthid'])
            _change_permissions_recursive(digital_output_path, 0o777)
            del dig_res_df

        # Storing day level data if required
        if day_level :
            dig_res_df_day = dig_temp_df.groupby(grouper_cols_day)[['pos_itm_tot_qt']].sum().reset_index()
            dig_res_df_day = dig_res_df_day.merge(dig_temp_df.groupby(grouper_cols_day)[['terr_pos_trn_typ_cd',
                                                                        'pos_busn_dt',
                                                                        'mcd_gbal_lcat_id_nu',
                                                                        'pos_ord_nu']].apply(add_gc_count).reset_index(), 
                                              on=grouper_cols_day, how='outer')

            # Storing result df    
            dig_res_df_day.rename(columns={0:'num_trans', 'pos_itm_tot_qt':'quantity'}, inplace=True)

            # create dir if donesn't exist 
            os.makedirs(digital_day_output_path, exist_ok=True)

            dig_res_df_day = format_dtype(dig_res_df_day, add_monthid=True)

            dig_res_df_day.to_parquet(digital_day_output_path, partition_cols=['monthid'] )
            _change_permissions_recursive(digital_day_output_path, 0o777)

            del dig_res_df_day

        # Clearing space
        del dig_temp_df
        gc.collect()

    # Loading and storing offline data
    if 'offline' in result_filter or 'all' in result_filter or 'both' in result_filter:

        # Defining variable paths            
        offline_output_path = cfg['UNITS'].format(**cfg, type_='offline')
        offline_day_output_path = cfg['DAY_UNITS'].format(**cfg, type_='offline')
        offline_wap_output_path = cfg['WAP'].format(**cfg, type_='offline')

        # Selecting offline trns and getting WAP and Mode data 
        off_temp_df = df.query('digital_flag.isna()').reset_index(drop=True)

        if get_price_info :

            # Getting WAP
            off_temp_df['mult'] = off_temp_df['pos_itm_grss_unt_prc_am']*off_temp_df['pos_itm_tot_qt']
            off_wap_df = off_temp_df.groupby(grouper_cols)[['mult', 'pos_itm_tot_qt']].sum().reset_index()
            off_wap_df['WAP'] = (off_wap_df['mult']/off_wap_df['pos_itm_tot_qt']).round(2)

            # Getting Mode
            off_md_df = off_temp_df.groupby(grouper_cols + ['pos_itm_grss_unt_prc_am'])[['pos_itm_tot_qt']].sum().reset_index()
            off_md_df['rank'] = off_md_df.groupby(grouper_cols)[['pos_itm_tot_qt']].rank(ascending=False)['pos_itm_tot_qt'].values
            off_md_df = off_md_df[(off_md_df['rank'] == 1)].reset_index(drop=True)
            off_md_df['mult'] = off_md_df['pos_itm_grss_unt_prc_am']*off_md_df['pos_itm_tot_qt']

            off_md_df_res = off_md_df.groupby(grouper_cols)[['mult', 'pos_itm_tot_qt']].sum().reset_index()
            off_md_df_res['Mode_price'] = off_md_df_res['mult']/off_md_df_res['pos_itm_tot_qt']

            del off_wap_df['mult']
            del off_wap_df['pos_itm_tot_qt']

            del off_md_df_res['mult']
            del off_md_df_res['pos_itm_tot_qt']

            # Doing processing and storing data            
            off_wap_df = off_wap_df.merge(off_md_df_res, on=grouper_cols, how='left')

            # create dir if doesn't exist 
            os.makedirs(offline_wap_output_path, exist_ok=True)
            off_wap_df = format_dtype(off_wap_df, add_monthid=True)

            off_wap_df.to_parquet(offline_wap_output_path, partition_cols=['sld_menu_itm_id', 'monthid'])
            _change_permissions_recursive(offline_wap_output_path, 0o777)

            # Clearing spaces
            del off_wap_df
            del off_md_df


        if item_level:

            # Getting GC and WAP data
            off_res_df = off_temp_df.groupby(grouper_cols)[['pos_itm_tot_qt']].sum().reset_index()
            off_res_df = off_res_df.merge(off_temp_df.groupby(grouper_cols)[['terr_pos_trn_typ_cd',
                                                                        'pos_busn_dt',
                                                                        'mcd_gbal_lcat_id_nu',
                                                                        'pos_ord_nu']].apply(add_gc_count).reset_index(), 
                                      on=grouper_cols, how='outer')

            # create dir if donesn't exist 
            os.makedirs(offline_output_path, exist_ok=True)

            # Doing processing and storing result df
            off_res_df.rename(columns={0:'num_trans', 'pos_itm_tot_qt':'quantity'}, inplace=True)      
            off_res_df = format_dtype(off_res_df, add_monthid=True)

            off_res_df.to_parquet(offline_output_path, partition_cols=['sld_menu_itm_id', 'monthid'])
            _change_permissions_recursive(offline_output_path, 0o777)
            del off_res_df

        # Storing day level data if required
        if day_level :
            off_res_df_day = off_temp_df.groupby(grouper_cols_day)[['pos_itm_tot_qt']].sum().reset_index()
            off_res_df_day = off_res_df_day.merge(off_temp_df.groupby(grouper_cols_day)[['terr_pos_trn_typ_cd',
                                                                        'pos_busn_dt',
                                                                        'mcd_gbal_lcat_id_nu',
                                                                        'pos_ord_nu']].apply(add_gc_count).reset_index(), 
                                              on=grouper_cols_day, how='outer')

            # Storing result df
            off_res_df_day.rename(columns={0:'num_trans', 'pos_itm_tot_qt':'quantity'}, inplace=True)

            # create dir if donesn't exist 
            os.makedirs(offline_day_output_path, exist_ok=True)
            off_res_df_day = format_dtype(off_res_df_day, add_monthid=True)

            off_res_df_day.to_parquet(offline_day_output_path, partition_cols=['monthid'])
            _change_permissions_recursive(offline_day_output_path, 0o777)
            del off_res_df_day

        # Clearing spoces
        del off_temp_df
        gc.collect()


    # Storing data for overall level
    if 'all' in result_filter or 'overall' in result_filter :

        # Defining variable paths
        overall_output_path = cfg['UNITS'].format(**cfg, type_='overall')
        overall_day_output_path = cfg['DAY_UNITS'].format(**cfg, type_='overall')
        overall_wap_output_path = cfg['WAP'].format(**cfg, type_='overall')

        if get_price_info :
            # Getting WAP and mode data
            df['mult'] = df['pos_itm_grss_unt_prc_am']*df['pos_itm_tot_qt']
            wap_df = df.groupby(grouper_cols)[['mult', 'pos_itm_tot_qt']].sum().reset_index()
            wap_df['WAP'] = (wap_df['mult']/wap_df['pos_itm_tot_qt']).round(2)

            # Getting Mode
            md_df = df.groupby(grouper_cols + ['pos_itm_grss_unt_prc_am'])[['pos_itm_tot_qt']].sum().reset_index()
            md_df['rank'] = md_df.groupby(grouper_cols)[['pos_itm_tot_qt']].rank(ascending=False)['pos_itm_tot_qt'].values
            md_df = md_df[(md_df['rank'] == 1)].reset_index(drop=True)
            md_df['mult'] = md_df['pos_itm_grss_unt_prc_am']*md_df['pos_itm_tot_qt']

            md_df_res = md_df.groupby(grouper_cols)[['mult', 'pos_itm_tot_qt']].sum().reset_index()
            md_df_res['Mode_price'] = md_df_res['mult']/md_df_res['pos_itm_tot_qt']

            del wap_df['mult']
            del wap_df['pos_itm_tot_qt']

            del md_df_res['mult']
            del md_df_res['pos_itm_tot_qt']

            # Doing processing and storing data
            wap_df = wap_df.merge(md_df_res, on=grouper_cols, how='left')

            # create dir if doesn't exist 
            os.makedirs(overall_wap_output_path, exist_ok=True)
            wap_df = format_dtype(wap_df, add_monthid=True)

            wap_df.to_parquet(overall_wap_output_path, partition_cols=['sld_menu_itm_id', 'monthid'])
            _change_permissions_recursive(overall_wap_output_path, 0o777)

            # Clearing spaces
            del wap_df
            del md_df

        if item_level :

            # Getting GC and Units data
            res_df = df.groupby(grouper_cols)[['pos_itm_tot_qt']].sum().reset_index()
            res_df = res_df.merge(df.groupby(grouper_cols)[['terr_pos_trn_typ_cd',
                                                                        'pos_busn_dt',
                                                                        'mcd_gbal_lcat_id_nu',
                                                                        'pos_ord_nu']].apply(add_gc_count).reset_index(), 
                                      on=grouper_cols, how='outer')

            # create dir if doesn't exist 
            os.makedirs(overall_output_path, exist_ok=True)

            # Doing processing and storing result df
            res_df.rename(columns={0:'num_trans', 'pos_itm_tot_qt':'quantity'}, inplace=True)            
            res_df = format_dtype(res_df, add_monthid=True)

            res_df.to_parquet(overall_output_path, partition_cols=['sld_menu_itm_id', 'monthid'])
            _change_permissions_recursive(overall_output_path, 0o777)


        # Storing day level data if required
        if day_level :
            res_df_day = df.groupby(grouper_cols_day)[['pos_itm_tot_qt']].sum().reset_index()
            res_df_day = res_df_day.merge(df.groupby(grouper_cols_day)[['terr_pos_trn_typ_cd',
                                                                        'pos_busn_dt',
                                                                        'mcd_gbal_lcat_id_nu',
                                                                        'pos_ord_nu']].apply(add_gc_count).reset_index(), 
                                              on=grouper_cols_day, how='outer')

            # Storing result df
            res_df_day.rename(columns={0:'num_trans', 'pos_itm_tot_qt':'quantity'}, inplace=True)


            # create dir if donesn't exist 
            os.makedirs(overall_day_output_path, exist_ok=True)
            res_df_day = format_dtype(res_df_day, add_monthid=True)

            res_df_day.to_parquet(overall_day_output_path, partition_cols=['monthid'])
            _change_permissions_recursive(overall_day_output_path, 0o777)


    # Clearing spaces
    del df
    gc.collect()

    print(file, ' is completed')
    #return 1
except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    print(exc_type, fname, exc_tb.tb_lineno)
    #return e

month_id=20210801
abc


NameError: name 'sys' is not defined

In [39]:
cmbo_df

,pos_busn_dt,mcd_gbal_lcat_id_nu,pos_ord_nu,cmbo_pren_sld_menu_itm_id,pos_itm_grss_unt_prc_am,sld_menu_itm_id,terr_pos_trn_typ_cd,pos_itm_tot_qt,pos_itm_grss_unt_prc_am
0,2021-08-01,195500017983,893237765,8825,97.00,8825,000,1,146.00
1,2021-08-01,195500756684,576685983,8827,216.00,8827,100,1,216.00
2,2021-08-01,195500753799,1006186365,8826,87.00,8826,000,1,87.00
3,2021-08-01,195500188058,697564480,8865,140.00,8865,100,1,189.00
4,2021-08-01,195500652290,957559262,8869,106.00,8869,000,1,106.00
...,...,...,...,...,...,...,...,...,...
56,2021-08-29,195500030791,236869168,1702,0.00,4900,105,1,1.06
57,2021-08-29,195500552231,270986641,1500,5.61,1500,105,1,5.61
58,2021-08-29,195500196381,1005724657,1701,2.28,1701,105,1,2.63
59,2021-08-29,195500552231,270986641,7024,0.00,4901,105,1,7.54


In [40]:
alc_df

,pos_busn_dt,mcd_gbal_lcat_id_nu,pos_ord_nu,cmbo_pren_sld_menu_itm_id,pos_itm_grss_unt_prc_am,sld_menu_itm_id,terr_pos_trn_typ_cd,pos_itm_tot_qt
0,2021-08-01,195500188058,278105123,-1,79.00,7617,100,1
1,2021-08-01,195500768700,900228418,-1,95.00,9350,000,1
2,2021-08-01,195500886008,698237170,-1,90.00,8911,100,1
3,2021-08-01,195500188058,278105123,-1,77.00,3030,100,1
5,2021-08-01,195500531533,306579944,-1,259.00,2044,000,1
...,...,...,...,...,...,...,...,...
190,2021-08-29,195500196381,1005724657,-1,0.00,7518,105,2
191,2021-08-29,195500030791,236869168,-1,0.00,3412,105,3
193,2021-08-29,195500694375,137690215,-1,0.75,6222,105,1
194,2021-08-29,195500694375,137690215,-1,0.00,3200,105,1


In [23]:
set(cmbo_df.columns) - set(alc_df.columns)

set()

In [32]:
result_list = []
def log_result(result):
    # This is called whenever foo_pool(i) returns a result.
    # result_list is modified only by the main process, not the pool workers.
    result_list.append(result)

In [33]:
import os
os.environ['NUM_THREADS']='1'
os.environ['OMP_NUM_THREADS']='1'
os.environ['NUMEXPR_NUM_THREADS']='1'


In [4]:
%%writefile process_item_data.py
import pandas as pd
import numpy as np

import os
import gc

from ta_lib.data_utils.data_processing import get_aggregated_data
from ta_lib.core.api import load_yml, create_context

# Defining configs

config_path = '../conf/config.yml'
cfg = create_context(config_path)

result_list = []
def log_result(result):
    # This is called whenever foo_pool(i) returns a result.
    # result_list is modified only by the main process, not the pool workers.
    result_list.append(result)

dates_=sorted(pd.date_range('2018-01-01','2021-08-01',freq='MS'),reverse=True)
dates_=[i.strftime('%Y%m%d') for i in dates_]

pool = mlp.Pool(11)
for run_date in dates_:
#     pool.apply_async(, args=(cfg.offline, result_filter='all', day_level=True, item_level=True, get_price_info=False,include_105=True), callback = log_result)
    pool.apply_async(get_aggregated_data.get_ungrouped_data, args = (cfg.offline, 'all', True, True, False,True), callback = log_result)
pool.close()
pool.join()

Writing process_item_data.py


In [15]:
(44*3*5)/4

165.0

In [16]:
44/4

11.0

In [7]:
import time
t1 = time.time()
get_aggregated_data.get_grouped_units_from_ungrouped(config.offline,'digital')
print("time taken : ", (time.time()-t1)/60)

time taken :  8.655671322345734


In [10]:
df_new=pd.read_parquet('/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/digital/grouped/item_level/sld_menu_itm_id=2100')
df_old=pd.read_parquet('/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/digital/grouped/item_level_bkp/sld_menu_itm_id=2100')

In [14]:
df_temp=df_new[df_new.pos_busn_dt>'2021-08-01'].merge(df_old[df_old.pos_busn_dt>'2021-08-01'],suffixes=('_new','_old'),on=['pos_busn_dt'])

In [16]:
df_temp['per']=df_temp['quantity_old']/df_temp['quantity_new']

In [17]:
df_temp

,pos_busn_dt,quantity_new,monthid_new,quantity_old,num_trans,monthid_old,per
0,2021-08-02,117969,202108,97137,65645,202108,0.823411
1,2021-08-03,123881,202108,102601,69552,202108,0.828222
2,2021-08-04,116045,202108,95331,64434,202108,0.821500
3,2021-08-05,121851,202108,100165,67456,202108,0.822029
4,2021-08-06,126185,202108,102212,68275,202108,0.810017
5,2021-08-07,121756,202108,98725,64327,202108,0.810843
6,2021-08-08,122749,202108,98903,64216,202108,0.805734
7,2021-08-09,8740,202108,8014,6687,202108,0.916934
8,2021-08-10,9042,202108,8283,6880,202108,0.916058
9,2021-08-11,8881,202108,8062,6767,202108,0.907781


In [19]:
# df_105=pd.read_parquet('/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/raw/gdw_data_105_trn/month_id=20210801')
df_dig=pd.read_parquet('/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/raw/digital/trans_ids/month_id=20210801')

In [20]:
df_dig['flag']=1
df_dig=df_dig.drop_duplicates()

In [21]:
df_dig_missed=df_105.merge(df_dig,on=['pos_ord_nu','mcd_gbal_lcat_id_nu'])

In [22]:
missed_qty=df_dig_missed.query('sld_menu_itm_id==2100').groupby('pos_busn_dt')['pos_itm_tot_qt'].sum()

In [23]:
missed_qty

pos_busn_dt
2021-08-01     1784
2021-08-02    20832
2021-08-03    21280
2021-08-04    20714
2021-08-05    21686
2021-08-06    23973
2021-08-07    23031
2021-08-08    23846
2021-08-09      726
2021-08-10      759
2021-08-11      819
2021-08-12      883
2021-08-13      966
2021-08-14      986
2021-08-15     1076
2021-08-16      796
2021-08-17      809
2021-08-18      841
2021-08-19      809
2021-08-20     1049
2021-08-21     1167
2021-08-22     1146
2021-08-23      808
2021-08-24      887
2021-08-25      797
2021-08-26      860
2021-08-27      988
2021-08-28      989
2021-08-29      994
2021-08-30     1467
2021-08-31     1371
Name: pos_itm_tot_qt, dtype: int64

In [25]:
df_new=pd.read_parquet('/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/digital/ungrouped_data/item_level/sld_menu_itm_id=2100')
df_old=pd.read_parquet('/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/digital/ungrouped_data/item_level_bkp/sld_menu_itm_id=2100')

In [26]:
df_temp=df_new[df_new.pos_busn_dt>'2021-08-01'].merge(df_old[df_old.pos_busn_dt>'2021-08-01'],suffixes=('_new','_old'),on=['pos_busn_dt'])

In [28]:
df_final=df_temp.merge(missed_qty.reset_index(),on=['pos_busn_dt'])

In [30]:
df_final['qty_105']=df_final['pos_itm_tot_qt_x']+df_final['pos_itm_tot_qt_y']

In [31]:
df_final - () + 105 

,pos_busn_dt,quantity,num_trans,monthid_new,pos_itm_tot_qt_x,gc,monthid_old,pos_itm_tot_qt_y,qty_105
0,2021-08-02,117969,79165,202108,97137,65645,202108,20832,117969
1,2021-08-03,123881,83510,202108,102601,69552,202108,21280,123881
2,2021-08-04,116045,77843,202108,95331,64434,202108,20714,116045
3,2021-08-05,121851,81473,202108,100165,67456,202108,21686,121851
4,2021-08-06,126185,83572,202108,102212,68275,202108,23973,126185
5,2021-08-07,121756,78728,202108,98725,64327,202108,23031,121756
6,2021-08-08,122749,79067,202108,98903,64216,202108,23846,122749
7,2021-08-09,8740,7263,202108,8014,6687,202108,726,8740
8,2021-08-10,9042,7483,202108,8283,6880,202108,759,9042
9,2021-08-11,8881,7413,202108,8062,6767,202108,819,8881
